In [1]:
# Imports
import os
import subprocess
import sys

# REMOVE!!
from utils import *

# model

from ir_lmart import *

# HPO

from hpo import *
from HpoWorker import *

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS
from hpbandster.examples.commons import MyWorker


# In[2]:


# In[3]:

In [2]:
def get_train_budget_data_file(budget, query_list, train_data_file):
    # Budget is percentage of training data: 
    # min_budget = 10%
    # max_budget = 100%
    if (int(budget) <= 100 or int(budget) >= 10):
        len_queries = len(query_list)
        budgeted_queries = round(len_queries * (budget / 100))
        print('total budget:', len_queries)
        print('allocated budget:', budgeted_queries)
        train_budget_queries_file = train_data_file + '_budget' + str(budget)
        if not os.path.exists(train_budget_queries_file):
            with open(train_data_file, 'rt') as f_in:
                with open(train_budget_queries_file, 'wt') as budget_file_out:
                    for query_feature in f_in:                        
                        qid = query_feature.split()[1].split(':')[1]
#                         print(qid)
                        if qid in query_list[0:budgeted_queries]:
                            
                            budget_file_out.write(query_feature)
        else:
            print("File already exists")
            return train_budget_queries_file                
    else:
        print('Budget is outside the limits (10% < b < 100%): ', budget)
        return 

In [3]:
# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        
# In[4]:

In [4]:
class fakeParser:
    def __init__(self):
        self.min_budget = 20 
        self.max_budget = 20
        self.n_iterations = 1
        self.n_workers = 1
        

In [5]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=int, help='Minimum budget used during the optimization.',    default=10)
    parser.add_argument('--max_budget',   type=int, help='Maximum budget used during the optimization.',    default=100)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)

#     args=parser.parse_args()
    args = fakeParser()
    
#     dataset = sys.argv[1] # 'bioasq'
#     workdir = './' + dataset + '_dir/'
#     data_split = sys.argv[2] # 'test'

    dataset = 'bioasq'
    workdir = './' + dataset + '_dir/'
    data_split =  'train'
    k_fold = 's1' 
    ranklib_location = '../../../ranklib/'
    
#     train_data_file = './bioasq_dir/bioasq.trai_features_reduced'
#     val_data_file = './bioasq_dir/bioasq.dev_features_reduced'
#     test_data_file = './bioasq_dir/bioasq.test_features_reduced'
    
    train_data_file = './bioasq_dir/bioasq_train_features'
    val_data_file = './bioasq_dir/bioasq_dev_features'
    test_data_file = './bioasq_dir/bioasq_test_features'
    
    l2r_model = '_lmart_'
    

In [6]:
    confdir = './' + dataset + '_config/'
    enabled_features_file = confdir + dataset + l2r_model + 'enabled_features'
    
#     print(enabled_features_file)
    # Train L2R model: LambdaMART
    # Parameters 
    
#     n_leaves = '10'
#     learning_rate = '0.1'
#     n_trees = '1000'
#     hpo_params = [n_leaves, learning_rate, n_trees]
    
    
    
    metric2t = 'MAP' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'
    
    ranker_type = '6' # LambdaMART
    
    # normalization: Feature Engineering?
    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    l2r_params = [
        '-validate',
        val_data_file,
        '-ranker',
        ranker_type,
        '-metric2t',
        metric2t,
        '-feature',
        enabled_features_file
    ]
    
    # Run train
    
#     lmart_model = L2Ranker(ranklib_location, l2r_params)
    lmart_model = L2Ranker(ranklib_location, l2r_params, norm_params)
    


# In[7]:


#     lmart_model.train(train_data_file, hpo_params)


# In[8]:


#     lmart_model.gen_run_file(test_data_file, run_file)


# In[9]:

In [7]:
trec_eval_command = '../../eval/trec_eval'
qrels_val_file = './bioasq_dir/bioasq_dev_qrels'
#     eval(trec_eval_command, qrels_file, './run_l2linear')


# In[ ]:





# In[10]:


#     # HPO 

    
    
# #     worker = HpoWorker(run_id='0')
#     cs = worker.get_configspace()

#     config = cs.sample_configuration().get_dictionary()
    
        
# #     pre_run_file = workdir + 'pre_run_' + dataset + l2r_model
    
#     run_file = workdir + 'run_' + dataset + l2r_model
    
#     print(config)
#     res = worker.compute(config=config)
#     print(res['loss'])


# In[11]:

In [8]:
# Start a nameserver (see example_3)
NS = hpns.NameServer(run_id='example1', host='127.0.0.1', port=None)
NS.start()


# In[12]:

save_model_prefix = workdir + 'ejemplo_model'
run_file_prefix = workdir + 'run_ejemplo'

train_queries_file = '../../bioasq_data/bioasq.' + 'train' + '.json'
            
query_list = load_queries(train_queries_file)
qid_list = [q['id'] for q in query_list] 
# train_features_file =  fold_dir + dataset + '_' + 'train' + '_features'
train_features_file =  workdir + dataset + '_' + 'train' + '_features'


budget_train_features_file = train_data_file
# budget_train_features_file = workdir + 'budget_ejemplo'


workers=[]
for i in range(args.n_workers):
    worker = HpoWorker(lmart_model, save_model_prefix, run_file_prefix, budget_train_features_file, qid_list, trec_eval_command, 
                       qrels_val_file, nameserver='127.0.0.1',run_id='example1', id=i)
    worker.run(background=True)
    workers.append(worker)


# In[13]:


# Run an optimizer (see example_2)
bohb = BOHB(  configspace = worker.get_configspace(),
                      run_id = 'example1', 
                      min_budget = args.min_budget, max_budget = args.max_budget
               )
res = bohb.run(n_iterations = args.n_iterations, min_n_workers = args.n_workers)


# In[14]:


# Step 4: Shutdown
# After the optimizer run, we must shutdown the master and the nameserver.
bohb.shutdown(shutdown_workers=True)
NS.shutdown()


# In[15]:


# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()


# In[16]:


all_runs = res.get_all_runs()



print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in all_runs])/args.max_budget))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in all_runs])/args.max_budget))
print('The run took  %.1f seconds to complete.'%(all_runs[-1].time_stamps['finished'] - all_runs[0].time_stamps['started']))


# In[18]:


#     qrels_test_file = './bioasq_dir/bioasq.test_qrels'
#     run_val_file = './this.file'
#     lmart_model = res['info']
#     lmart_model.gen_run_file(test_data_file, run_val_file)
#     eval(trec_eval_command, qrels_test_file, run_val_file)


# In[10]:


id2config

total budget: 1751
allocated budget: 350
File already exists
budget_file: ./bioasq_dir/bioasq_train_features_budget20
Model saved:  ./bioasq_dir/ejemplo_model_leaves10_lr0.07_n110
Aqui veo si genero o no el run file:  <class 'list'>
['../../eval/trec_eval', '-m', 'map', '-m', 'P.20', '-m', 'ndcg_cut.20', './bioasq_dir/bioasq_dev_qrels', './bioasq_dir/run_ejemplo_leaves10_lr0.07_n110']
Run error:  None
Best found configuration: {'learning_rate': 0.07, 'n_leaves': 10, 'n_trees': 110}
A total of 1 unique configurations where sampled.
A total of 1 runs where executed.
Total budget corresponds to 1.0 full function evaluations.
Total budget corresponds to 1.0 full function evaluations.
The run took  6.5 seconds to complete.


{(0, 0, 0): {'config': {'learning_rate': 0.07, 'n_leaves': 10, 'n_trees': 110},
  'config_info': {'model_based_pick': False}}}